In [1]:
#  TensorFlow Tutorials / ロジスティック回帰

import tensorflow as tf
import numpy as np
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split

# weights 初期化のサポート関数
def init_weights(shape):
    return tf.Variable(tf.random_normal(shape, stddev=0.01))

# オリジナルコードでは，バイアスをmodelに入れていないが，本コードは入れる．
def model(X, w, b):
    return tf.matmul(X, w) + b

# digits は，scikit-learn が用意する手書き文字データセット．pixel: 8 x 8
def load_data():
    digits = load_digits()
    digits_images = digits.data / 16.   # scaling to (0 .. 1)
    digits_target_ = []
    for i in range(len(digits.target)):
        target_one = np.zeros([10], dtype=np.float32)
        target_one[digits.target[i]] = 1.
        digits_target_.append(target_one)
    digits_target_onehot = np.asarray(digits_target_)
    return digits_images, digits_target_onehot

X, Y = load_data()

# train / test セットに分割する
trX, teX, trY, teY =  train_test_split(X, Y, test_size=0.2)

# TensorFlow placeholders
X_ph = tf.placeholder(tf.float32, [None, 64])
Y_ph = tf.placeholder(tf.float32, [None, 10])

# オリジナルコードでは，バイアスをmodelに入れていないが，本コードでは入れる．
w = init_weights([64, 10])
b = tf.zeros([10])

py_x = model(X_ph, w, b)
epoch = 120

# コスト関数，オプティマイザ，予測
cost = tf.nn.softmax_cross_entropy_with_logits(logits=py_x, labels=Y_ph)
train_op = tf.train.GradientDescentOptimizer(0.01).minimize(cost)
predict_op = tf.argmax(py_x, 1)

# TensorFlow session
with tf.Session() as sess:
    # you need to initialize all variables
    tf.global_variables_initializer().run()

    for i in range(epoch):
        for start, end in zip(range(0, len(trX), 128), range(128, len(trX)+1, 128)):
            train_feed = {X_ph: trX[start:end], Y_ph: trY[start:end]}
            sess.run(train_op, feed_dict=train_feed)
        
        # numpy レベルで正解率(accuracy)を算定し出力
        if i % 10 == 0:
            print('step {:>3d}: accuracy = {:>8.3f}'.format(
                i, np.mean(np.argmax(teY, axis=1) ==
                         sess.run(predict_op, feed_dict={X_ph: teX}))))


step   0: accuracy =    0.808
step  10: accuracy =    0.931
step  20: accuracy =    0.936
step  30: accuracy =    0.939
step  40: accuracy =    0.944
step  50: accuracy =    0.947
step  60: accuracy =    0.947
step  70: accuracy =    0.950
step  80: accuracy =    0.950
step  90: accuracy =    0.947
step 100: accuracy =    0.947
step 110: accuracy =    0.947
step 120: accuracy =    0.950
